# Create Amazon Omics Analytic Stores to Import Genomic Data and Run Queries

### Follow steps in this notebook to: 1) create Amazon Omics Reference, Variant, and Annotation Stores; 2) import reference genome, variant files, and ClinVar annotation file from S3 to the respective data stores; 3) query the variant and annotation data. 

## Create reference store and import reference genome

In [1]:
from datetime import datetime
import itertools as it
import json
import gzip
import os
from pprint import pprint
import time
import urllib

import boto3
import botocore.exceptions
import requests

### Create service role

In [2]:
# set a timestamp
dt_fmt = '%Y%m%dT%H%M%S'
ts = datetime.now().strftime(dt_fmt)

policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "omics:*"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "ram:AcceptResourceShareInvitation",
        "ram:GetResourceShareInvitations"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "s3:GetBucketLocation",
        "s3:PutObject",
        "s3:GetObject",
        "s3:ListBucket",
        "s3:AbortMultipartUpload",
        "s3:ListMultipartUploadParts",
        "s3:GetObjectAcl",
        "s3:PutObjectAcl"
      ],
      "Resource": "*"
    }
  ]
}

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "omics.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

In [3]:
# Base name for role and policy
omics_iam_name = f'multimodal-omics-{ts}'

# Create the IAM client
iam = boto3.resource('iam')

# Check if the role already exist. If not, create it
try:
    role = iam.Role(omics_iam_name)
    role.load()
    
except botocore.exceptions.ClientError as ex:
    if ex.response["Error"]["Code"] == "NoSuchEntity":
        #Create the role with the corresponding trust policy
        role = iam.create_role(
            RoleName=omics_iam_name, 
            AssumeRolePolicyDocument=json.dumps(trust_policy))
        
        #Create policy
        policy = iam.create_policy(
            PolicyName='{}-policy'.format(omics_iam_name), 
            Description="Policy for Amazon Omics",
            PolicyDocument=json.dumps(policy))
        
        #Attach the policy to the role
        policy.attach_role(RoleName=omics_iam_name)
    else:
        print (ex)
        print('Somthing went wrong, please retry and check your account settings and permissions')

In [4]:
def get_role_arn(role_name):
    try:
        iam = boto3.resource('iam')
        role = iam.Role(role_name)
        role.load()  # calls GetRole to load attributes
    except botocore.exeptions.ClientError:
        print("Couldn't get role named %s."%role_name)
        raise
    else:
        return role.arn

In [171]:
# Replace <S3 URI for FASTA file> with the S3 URI of the reference genome in FASTA format. 

SOURCE_S3_URIS = {
    "reference": "<S3 URI for FASTA file>"
    }

### Create Omics client

In [6]:
omics = boto3.client('omics', region_name='us-east-1')

### Create reference store 

In [8]:
def get_ref_store_id(client=None):
    if not client:
        client = boto3.client('omics')
    
    resp = client.list_reference_stores(maxResults=10)
    list_of_stores = resp.get('referenceStores')
    store_id = None
    
    if list_of_stores != None:
        # Since there can only be one store per region, if there is a store present use the first one
        store_id = list_of_stores[0].get('id')
    
    return store_id

In [ ]:
print(f"Checking for a reference store in region: {omics.meta.region_name}")
if get_ref_store_id(omics) == None:
    response = omics.create_reference_store(name='myReferenceStore')
    print(response)
else:
    print("Congratulations, you have an existing reference store!")

### Import reference genome to reference store

In [10]:
# If using a different reference genomem, replace "hg19" with a different prefix

ref_name = f'hg19-{ts}'

ref_import_job = omics.start_reference_import_job(
    referenceStoreId=get_ref_store_id(omics), 
    roleArn=get_role_arn(omics_iam_name),
    sources=[{
        'sourceFile': SOURCE_S3_URIS["reference"],
        'name': ref_name,
        'tags': {'SourceLocation': '1kg'}
    }])

In [ ]:
ref_import_job = omics.get_reference_import_job(
    referenceStoreId=get_ref_store_id(omics), 
    id=ref_import_job['id'])
ref_import_job

### !!! Wait until the above import job has finished !!!

In [ ]:
resp = omics.list_references(referenceStoreId=get_ref_store_id(omics), filter={"name": ref_name})

ref_list = resp
pprint(resp)

In [ ]:
# Store this reference
ref = omics.get_reference_metadata(
    referenceStoreId=get_ref_store_id(omics), 
    id=ref_list['references'][0]['id'])
ref

## Create Variant Store and import VCF files

In [128]:
def get_reference_arn(ref_name, client=None):
    if not client:
        client = boto3.client('omics')
    
    resp = client.list_reference_stores(maxResults=10)
    ref_stores = resp.get('referenceStores')
    
    # There can only be one reference store per account per region
    # if there is a store present, it is the first one
    ref_store = ref_stores[0] if ref_stores else None
    
    if not ref_store:
        raise RuntimeError("You have not created a reference store, please got to the Amazon Omics Storage tutorial to learn how to create one. Do not continue with this notebook")
        
    ref_arn = None
    resp = omics.list_references(referenceStoreId=ref_store['id'])
    ref_list = resp.get('references')
    
    for ref in resp.get('references'):
        if ref['name'] == ref_name:
            ref_arn = ref['arn']
    
    if ref_arn == None:
        raise RuntimeError(f"Could not find {ref_name}.")
    
    return ref_arn

In [130]:
# Replace <S3 URI for VCF files> with the S3 URI of the bucket containing the VCF files. 

SOURCE_VARIANT_URI = "<S3 URI for VCF files>"

In [131]:
source = urllib.parse.urlparse(SOURCE_VARIANT_URI)
bucket = source.netloc
prefix = source.path[1:]

s3r = boto3.resource('s3')

bucket = s3r.Bucket(bucket)
objects = bucket.objects.filter(Prefix=prefix, MaxKeys=10_000)
ext = '.vcf'

vcf_list = [f"s3://{o.bucket_name}/{o.key}" for o in objects if o.key.endswith(ext)]

### Create Variant Store

In [ ]:
var_store_name = f'synthea_newvariants_{ts.lower()}'

# Replace <reference name> with reference genome name created in the previous steps
ref_name = '<reference name>' 

response = omics.create_variant_store(
    name=var_store_name, 
    reference={"referenceArn": get_reference_arn(ref_name, omics)}
)

var_store = response
response

### !!! Wait until the Variant Store is created !!!

In [ ]:
try:
    waiter = omics.get_waiter('variant_store_created')
    waiter.wait(name=var_store['name'])

    print(f"variant store {var_store['name']} ready for use")
except botocore.exceptions.WaiterError as e:
    print(f"variant store {var_store['name']} FAILED:")
    print(e)

var_store = omics.get_variant_store(name=var_store['name'])

### Import VCF files

In [135]:
l_vcf = [dict(zip(["source"],[uri])) for i, uri in enumerate(vcf_list)]

response = omics.start_variant_import_job(destinationName=var_store['name'], 
                                          roleArn=get_role_arn(omics_iam_name),
                                          items=l_vcf)

## Query Variant Store with Amazon Athena

In [ ]:
# To run Athena queries on the data, use AWS LakeFormation to create resource links to the database
# and ensure IAM user running this notebook is a Data Lake Administrator.

ram = boto3.client('ram')
glue = boto3.client('glue')

caller_identity = boto3.client('sts').get_caller_identity()
AWS_ACCOUNT_ID = caller_identity['Account']
AWS_IDENITY_ARN = caller_identity['Arn']

In [ ]:
response = ram.list_resources(resourceOwner='OTHER-ACCOUNTS', resourceType='glue:Database')

if not response.get('resources'):
    print('no shared resources found. verify that you have successfully created an Omics Analytics store')
else:
    variantstore_resources = [resource for resource in response['resources'] if var_store['id'] in resource['arn']]
    if not variantstore_resources:
        print(f"no shared resources matching variant store id {var_store['id']} found")
    else:
        variantstore_resource = variantstore_resources[0]

variantstore_resource

resource_share = ram.get_resource_shares(
    resourceOwner='OTHER-ACCOUNTS', 
    resourceShareArns=[variantstore_resource['resourceShareArn']])['resourceShares'][0]
resource_share

# this creates a resource link to the table for the variant store and adds it to the `omicsdb` database
glue.create_table(
    DatabaseName='omicsdb',
    TableInput = {
        "Name": var_store['name'],
        "TargetTable": {
            "CatalogId": resource_share['owningAccountId'],
            "DatabaseName": f"variant_{AWS_ACCOUNT_ID}_{var_store['id']}",
            "Name": var_store['name'],
        }
    }
)

In [ ]:
# Use Athena engine version 3 to query Omics Analytic Stores 

athena = boto3.client('athena')

athena_workgroups = athena.list_work_groups()['WorkGroups']

athena_workgroup = None
for wg in athena_workgroups:
    print(wg['EngineVersion']['EffectiveEngineVersion'])
    if wg['EngineVersion']['EffectiveEngineVersion'] == 'Athena engine version 3':
        print(f"Workgroup '{wg['Name']}' found using Athena engine version 3")
        athena_workgroup = wg
        break
else:
    print("No workgroups with Athena engine version 3 found. creating one")
    athena_workgroup = athena.create_work_group(
        Name='omics',
        Configuration={
            "EngineVersion": {
                "SelectedEngineVersion": "Athena engine version 3"
            }
        }
    )

athena_workgroup

In [192]:
# Use AWS Wrangler to submit query and get results as a Pandas Dataframe

import awswrangler as wr

# Replace <variant-table> with the name of the annotation table created in the previous step

df_var = wr.athena.read_sql_query("select sampleid, contigname, start, referenceallele, alternatealleles, calls from <variant-table> limit 10;", 
                              database="omicsdb", workgroup = "omics")
df_var

,sampleid,contigname,start,referenceallele,alternatealleles,calls
0,3dcf722a-8e47-dd42-a0f3-97496f4e8d09,1,46932823,A,[G],"[0, 1]"
1,2a19fd41-040b-e9fb-fb5c-e9095f04922e,1,46932823,A,[G],"[0, 1]"
2,b3eaed13-18ab-509f-f9c7-2d0337181c7b,1,46932823,A,[G],"[0, 1]"
3,08dc48f1-2321-362b-8c26-9ec0ed06f4fd,1,46932823,A,[G],"[0, 1]"
4,a870a2c3-9fd9-7141-6391-b1428fb920bb,1,46932823,A,[G],"[0, 1]"
5,1acc5aa3-ba78-a290-7cf6-75284343ab22,1,46932823,A,[G],"[0, 1]"
6,1d08cfdb-2a91-55bd-4e27-2e88630849f3,1,46932823,A,[G],"[0, 1]"
7,6df219b3-88cd-b76a-2c14-40a6399152aa,1,46932823,A,[G],"[0, 1]"
8,7a4dad5f-3b06-3104-8540-f20982d2c4c5,1,46932823,A,[G],"[0, 1]"
9,30fe95e3-951f-ef8c-4b1d-c921a9a6dbdb,1,46932823,A,[G],"[0, 1]"


## Create Annotation Store and import ClinVar annotation file

In [ ]:
# Replace <S3 URI for annotation file> with the S3 URI of the ClinVar annotation file 

SOURCE_ANNOTATION_URI = "<S3 URI for annotation file>"

ann_store_name = f'synthea_annotations_{ts.lower()}'

# Replace <reference name> with reference genome name created in the previous steps
ref_name = '<reference name>' 

response = omics.create_annotation_store(
    name=ann_store_name, 
    reference={"referenceArn": get_reference_arn(ref_name, omics)},
    storeFormat='VCF'
)

ann_store = response
response

try:
    waiter = omics.get_waiter('annotation_store_created')
    waiter.wait(name=ann_store['name'])

    print(f"annotation store {ann_store['name']} ready for use")
except botocore.exceptions.WaiterError as e:
    print(f"annotation store {ann_store['name']} FAILED:")
    print(e)

ann_store = omics.get_annotation_store(name=ann_store['name'])


### !!! Wait until the Annotation Store is created !!!

In [88]:
source = urllib.parse.urlparse(SOURCE_ANNOTATION_URI)
bucket = source.netloc
prefix = source.path[1:]

s3r = boto3.resource('s3')

bucket = s3r.Bucket(bucket)
objects = bucket.objects.filter(Prefix=prefix, MaxKeys=10_000)
ext = '.vcf'

annotation_list = [f"s3://{o.bucket_name}/{o.key}" for o in objects if o.key.endswith(ext)]

### Import annotation file

In [ ]:
response = omics.start_annotation_import_job(
    destinationName=ann_store['name'],
    roleArn=get_role_arn(omics_iam_name),
    items=[{"source": SOURCE_ANNOTATION_URI}]
)
response

## Query Annotation Store with Amazon Athena

In [ ]:
# To run Athena queries on the data, use AWS LakeFormation to create resource links to the database and 

response = ram.list_resources(resourceOwner='OTHER-ACCOUNTS', resourceType='glue:Database')

if not response.get('resources'):
    print('no shared resources found. verify that you have successfully created an Omics Analytics store')
else:
    annotationstore_resources = [resource for resource in response['resources'] if ann_store['id'] in resource['arn']]
    if not annotationstore_resources:
        print(f"no shared resources matching annotation store id {ann_store['id']} found")
    else:
        annotationstore_resource = annotationstore_resources[0]

        resource_share = ram.get_resource_shares(
            resourceOwner='OTHER-ACCOUNTS', 
            resourceShareArns=[annotationstore_resource['resourceShareArn']])['resourceShares'][0]
        
        # this creates a resource link to the table for the annotation store and adds it to the `omicsdb` database
        glue.create_table(
            DatabaseName='omicsdb',
            TableInput = {
                "Name": ann_store['name'],
                "TargetTable": {
                    "CatalogId": resource_share['owningAccountId'],
                    "DatabaseName": f"annotation_{AWS_ACCOUNT_ID}_{ann_store['id']}",
                    "Name": ann_store['name'],
                }
            }
        )

In [189]:
# Replace <annotation-table> with the name of the annotation table created in the previous step

df_ann = wr.athena.read_sql_query("select contigname, start, referenceallele, alternatealleles, attributes from <annotation-table> order by contigname limit 10;", 
                              database="omicsdb", workgroup = "omics")
df_ann

,contigname,start,referenceallele,alternatealleles,attributes
0,1,926009,G,[T],"[(CLNSIG, Likely_benign), (GENEINFO, SAMD11:14..."
1,1,926026,C,[T],"[(CLNSIG, Likely_benign), (GENEINFO, SAMD11:14..."
2,1,925975,T,[C],"[(CLNSIG, Uncertain_significance), (GENEINFO, ..."
3,1,926002,C,[T],"[(CLNSIG, Uncertain_significance), (GENEINFO, ..."
4,1,926013,G,[A],"[(CLNSIG, Uncertain_significance), (GENEINFO, ..."
5,1,926024,G,[A],"[(CLNSIG, Likely_benign), (GENEINFO, SAMD11:14..."
6,1,925955,C,[T],"[(CLNSIG, Likely_benign), (GENEINFO, SAMD11:14..."
7,1,925968,C,[T],"[(CLNSIG, Likely_benign), (GENEINFO, SAMD11:14..."
8,1,925985,C,[T],"[(CLNSIG, Likely_benign), (GENEINFO, SAMD11:14..."
9,1,925951,G,[A],"[(RS, 1640863258), (CLNSIG, Uncertain_signific..."
